In [4]:
import oddt
from oddt import toolkit
from oddt.docking import AutodockVina
from tqdm.notebook import tqdm
import pickle
import pandas as pd
from rdkit import Chem

In [2]:
protein = next(oddt.toolkit.readfile('pdb', './8gcy_receptor_prep.pdb'))
protein.protein = True

In [3]:
ligand = next(oddt.toolkit.readfile('mol2', './8gcy_ligand_prep.mol2'))

In [4]:
test_docking = AutodockVina.autodock_vina(protein=protein, size=(20, 20,20), center=(2.208, 8.406, 18.575),
                                          exhaustiveness=8, num_modes=9, energy_range=3,
                                          executable='/usr/autodock_vina_1_1_2_linux_x86/bin/vina')

In [9]:
dockin_reslut = test_docking.dock(ligand)

In [10]:
len(dockin_reslut)

9

In [11]:
type(dockin_reslut)

list

In [12]:
dockin_reslut[0].data.keys()

['vina_affinity',
 'vina_rmsd_lb',
 'vina_rmsd_ub',
 'vina_rmsd_input',
 'vina_rmsd_input_min']

In [14]:
for result in dockin_reslut:
    print(result.data['vina_affinity'])

-11.3
-10.4
-10.1
-9.3
-8.8
-8.8
-8.5
-8.3
-8.3


In [4]:
protein = next(oddt.toolkit.readfile('pdb', './8gcy_receptor_prep.pdb'))
protein.protein = True
sdf_file = 'CBLB_inhibitors_vsF.sdf'
dock_conf = AutodockVina.autodock_vina(protein=protein, size=(20, 20,20), center=(2.208, 8.406, 18.575),
                                       exhaustiveness=8, num_modes=9, energy_range=3, n_cpu=8,
                                       executable='/usr/autodock_vina_1_1_2_linux_x86/bin/vina')

docking_results = []
for mol in tqdm(oddt.toolkit.readfile('sdf', sdf_file)):
    dock_result = dock_conf.dock(mol)
    docking_results.append(dock_result)

with open('./docking_results.pickle', 'wb') as fp:   #Pickling
    pickle.dump(docking_results, fp)

0it [00:00, ?it/s]

total_time 11742.967827796936


In [ ]:
# idx_affinity = [] # tuple, (idx of molecule in sdf file, best vina_affinity)
# for i, result in enumerate(docking_results):
#     aff = result[0].data['vina_affinity']
#     idx_affinity.append(i, aff)


In [5]:
sdf_file = 'CBLB_inhibitors_vsF.sdf'

with open('./docking_results.pickle', 'rb') as p:
    docking_results = pickle.load(p)

# vina_affinityのtopを取り出す
best_affinities = []
for result in docking_results:
    affinity = result[0].data['vina_affinity']
    best_affinities.append(affinity)
    
# SDFから化合物を読み込み
suppl = Chem.SDMolSupplier(sdf_file)

# SMILESのリスト作成
ligand_smis = [Chem.MolToSmiles(mol) for mol in suppl if mol is not None]

# dataframeにしてcsvを出力
df = pd.DataFrame(list(zip(ligand_smis, best_affinities)), columns =['SMILES', 'vina_affinity'])
df.to_csv('docking_output.csv', index=False)